<a href="https://colab.research.google.com/github/elichen/aoc2016/blob/main/Day_11_Radioisotope_Thermoelectric_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
data = """The first floor contains a hydrogen-compatible microchip and a lithium-compatible microchip.
The second floor contains a hydrogen generator.
The third floor contains a lithium generator.
The fourth floor contains nothing relevant.""".split('\n')

In [67]:
data = """The first floor contains a polonium generator, a thulium generator, a thulium-compatible microchip, a promethium generator, a ruthenium generator, a ruthenium-compatible microchip, a cobalt generator, and a cobalt-compatible microchip.
The second floor contains a polonium-compatible microchip and a promethium-compatible microchip.
The third floor contains nothing relevant.
The fourth floor contains nothing relevant.""".split('\n')

In [68]:
import re

flrs = [[] for _ in range(4)]

pattern = r'\b[a-zA-Z]+(?:-[a-zA-Z]+)?(?:-compatible)? (?:generator|microchip)\b'
for i in range(4):
  objects = re.findall(pattern, data[i])
  objects = [x.split()[0][:2]+x.split()[1][0] for x in objects]
  flrs[i] = set(objects)
flrs

[{'cog', 'com', 'pog', 'prg', 'rug', 'rum', 'thg', 'thm'},
 {'pom', 'prm'},
 set(),
 set()]

In [65]:
def check_move(src, dst, flrs):
  for i in [src,dst]:
    if len([x for x in flrs[i] if x.endswith("g")]) > 0:
      for c in [x for x in flrs[i] if x.endswith("m")]:
        if c[:2]+"g" not in flrs[i]:
          return False
  return True

def bfs(flrs):
  next = [(0, 0, flrs)]
  seen = set()
  while len(next) > 0:
    step, curr, flrs = next.pop(0)
    if (curr,tuple([tuple(x) for x in flrs])) in seen: continue
    seen.add((curr,tuple([tuple(x) for x in flrs])))
    # print(step, curr, flrs)
    if all(len(x)==0 for x in flrs[:3]):
      return step
    rets = []
    for c1 in flrs[curr]:
      for c2 in flrs[curr]:
        to_move = {c1,c2}
        dsts = []
        if curr<3: dsts.append(curr+1)
        if curr>0: dsts.append(curr-1)
        for dst in dsts:
          new_flrs = flrs.copy()
          new_flrs[curr] = new_flrs[curr] - to_move
          new_flrs[dst] = new_flrs[dst] | to_move
          if check_move(curr, dst, new_flrs):
            next.append((step+1, dst, new_flrs))

bfs(flrs)

47

In [1]:
data = """The first floor contains a elerium generator, elerium-compatible microchip, dilithium generator, dilithium-compatible microchip, polonium generator, a thulium generator, a thulium-compatible microchip, a promethium generator, a ruthenium generator, a ruthenium-compatible microchip, a cobalt generator, and a cobalt-compatible microchip.
The second floor contains a polonium-compatible microchip and a promethium-compatible microchip.
The third floor contains nothing relevant.
The fourth floor contains nothing relevant.""".split('\n')

In [2]:
import re

flrs = [[] for _ in range(4)]

pattern = r'\b[a-zA-Z]+(?:-[a-zA-Z]+)?(?:-compatible)? (?:generator|microchip)\b'
for i in range(4):
  objects = re.findall(pattern, data[i])
  objects = [x.split()[0][:2]+x.split()[1][0] for x in objects]
  flrs[i] = set(objects)
flrs

[{'cog',
  'com',
  'dig',
  'dim',
  'elg',
  'elm',
  'pog',
  'prg',
  'rug',
  'rum',
  'thg',
  'thm'},
 {'pom', 'prm'},
 set(),
 set()]

In [6]:
from functools import lru_cache

@lru_cache(maxsize=None)
def check_move(i, flrs):
  if len([x for x in flrs[i] if x.endswith("g")]) > 0:
    for c in [x for x in flrs[i] if x.endswith("m")]:
      if c[:2]+"g" not in flrs[i]:
        return False
  return True

seen = set((0, tuple(map(frozenset, flrs))))
min_dist = [300]

@lru_cache(maxsize=None)
def dfs(curr, flrs):
  # print(len(seen), curr, flrs)
  if len(seen) >= min_dist[0]: return float("inf")
  if all(len(x)==0 for x in flrs[:3]):
    min_dist[0] = min(min_dist[0],len(seen))
    return 0

  res = [float('inf')]
  for c1 in flrs[curr]:
    for c2 in flrs[curr]:
      to_move = {c1,c2}
      dsts = []
      if curr<3 and (c1.endswith("g") or c2.endswith("g")): dsts.append(curr+1)
      if curr>0 and len(to_move) == 1: dsts.append(curr-1)
      for dst in dsts:
        new_flrs = list(flrs)
        new_flrs[curr] = new_flrs[curr] - to_move
        new_flrs[dst] = new_flrs[dst] | to_move
        if (dst,tuple(map(frozenset, new_flrs))) in seen: continue
        seen.add((dst,tuple(map(frozenset, new_flrs))))
        if check_move(curr ,tuple(map(frozenset, new_flrs))) and check_move(dst,tuple(map(frozenset, new_flrs))):
            res.append(1+dfs(dst, tuple(map(frozenset, new_flrs))))
        seen.remove((dst,tuple(map(frozenset, new_flrs))))
  return min(res)

dfs(0, tuple(map(frozenset, flrs)))

71